<a href="https://colab.research.google.com/github/it-ces/PUJ-churn/blob/main/Modeling-churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/it-ces/PUJ-churn.git

Cloning into 'PUJ-churn'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 43 (delta 7), reused 37 (delta 4), pack-reused 0
Receiving objects: 100% (43/43), 9.71 MiB | 10.38 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
cd PUJ-churn

/content/PUJ-churn


In [3]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
import seaborn as sns
import warnings as wrn
wrn.filterwarnings('ignore')
from preprocess import breakdown_vars
from preprocess import dummies_ohe
from preprocess import std_z
from preprocess import Xy
from GeneticNeuralNetwork import gaMLP_ragnarok
from NeuralNetwork import MLPclassifer
from ineq import Theil
from gentools import select_optimun_parameters
from GeneticNeuralNetwork import gaMLP_score
from NeuralNetwork import MLP

In [5]:
df = pd.read_excel("bank_churn.xlsx")

In [6]:
df.to_csv("Churn-data.csv")

In [7]:
df['churn'] = df['attrition_flag'].replace(['Existing Customer', 'Attrited Customer'], [0, 1])
df.drop(columns=['clientnum','avg_open_to_buy','attrition_flag','credit_limit'], inplace=True)
df['churn'].value_counts(normalize=True) # Unbalance data....

churn
0    0.83934
1    0.16066
Name: proportion, dtype: float64

In [8]:
cats, binaries, nonormal, normal  = breakdown_vars(df)

In [9]:
df =  dummies_ohe(df, cats)

In [10]:
X,y = Xy(df,'churn')

In [11]:
y.value_counts(normalize=True)

churn
0    0.83934
1    0.16066
Name: proportion, dtype: float64

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = True, random_state = 4406, stratify=y)

In [13]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=1234)
X_train, y_train = rus.fit_resample(X_train, y_train)

In [14]:
y_train.value_counts()

churn
0    1139
1    1139
Name: count, dtype: int64

In [15]:
from preprocess import standardize_X_test

X_test = standardize_X_test(X_train, X_test) # Apply the mean and std of X_test
X_train = std_z(nonormal + normal, X_train)

customer_age
dependent_count
months_on_book
total_relationship_count
months_inactive_12_mon
contacts_count_12_mon
total_revolving_bal
total_amt_chng_q4_q1
total_trans_amt
total_trans_ct
total_ct_chng_q4_q1
avg_utilization_ratio


In [16]:
X_train

,customer_age,dependent_count,months_on_book,total_relationship_count,months_inactive_12_mon,contacts_count_12_mon,total_revolving_bal,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,...,marital_status_Single,marital_status_Unknown,income_category_$40K - $60K,income_category_$60K - $80K,income_category_$80K - $120K,income_category_Less than $40K,income_category_Unknown,card_category_Gold,card_category_Platinum,card_category_Silver
0,-1.196062,-0.297743,0.001618,0.861565,-0.500782,-1.446373,0.427114,-1.024812,-0.007007,0.683745,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2.117951,-1.842749,1.653669,0.861565,1.545461,0.290037,-0.276298,0.522915,0.039261,0.372932,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.206021,1.247264,0.255780,-0.412880,-0.500782,-0.578168,0.573190,0.935945,-0.599037,-0.870318,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-1.450986,-1.842749,0.001618,0.861565,-0.500782,-1.446373,0.604653,0.681773,-0.582420,-0.381898,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.941138,-0.297743,-1.777515,0.861565,-0.500782,-0.578168,-1.092076,0.000954,0.109966,1.571781,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,1.353179,-0.297743,0.637022,-0.412880,0.522340,0.290037,-1.092076,-0.026278,-0.542669,-0.959122,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2274,0.333483,-0.297743,0.255780,-0.412880,1.545461,0.290037,-1.092076,-1.147360,-0.739469,-0.959122,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2275,0.078559,0.474761,0.382860,-1.050103,0.522340,-0.578168,-1.092076,0.236971,-0.345217,-0.204291,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.333483,-0.297743,0.891184,-1.050103,0.522340,0.290037,1.064230,-1.065661,-0.113878,-0.293095,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
from models import grid_lr
from models import grid_SVM
from models import grid_MLP
from sklearn.metrics import classification_report

In [18]:
model_lr = grid_lr(X_train, y_train)
lr_predict = model_lr.predict(X_test)
print(classification_report(y_test, lr_predict))
lr_table  = pd.DataFrame(classification_report(y_test, lr_predict, output_dict=True)).iloc[:,0:2]

              precision    recall  f1-score   support

           0       0.97      0.84      0.90      2551
           1       0.51      0.87      0.65       488

    accuracy                           0.85      3039
   macro avg       0.74      0.86      0.77      3039
weighted avg       0.90      0.85      0.86      3039



In [19]:
model_SVM  = grid_SVM(X_train, y_train)
SVM_predict = model_SVM.predict(X_test)
print(classification_report(y_test, SVM_predict))
SVM_table  = pd.DataFrame(classification_report(y_test,  SVM_predict, output_dict=True)).iloc[:,0:2]


              precision    recall  f1-score   support

           0       0.99      0.89      0.94      2551
           1       0.62      0.94      0.75       488

    accuracy                           0.90      3039
   macro avg       0.80      0.91      0.84      3039
weighted avg       0.93      0.90      0.91      3039



In [20]:
model_mlp  = grid_MLP(X_train, y_train)
NN_predict = model_mlp.predict(X_test)
print(classification_report(y_test, NN_predict))
MLP_table  = pd.DataFrame(classification_report(y_test,  NN_predict, output_dict=True)).iloc[:,0:2]

              precision    recall  f1-score   support

           0       0.99      0.88      0.93      2551
           1       0.60      0.94      0.73       488

    accuracy                           0.89      3039
   macro avg       0.80      0.91      0.83      3039
weighted avg       0.93      0.89      0.90      3039



In [ ]:
architecture = (5,5,1)
Pop_sizes  = [55]             # 75
generations = [80]
Mutations = [0.05]  #0.05, 0.09
Tournaments = [5] # 3,5
Cross = [0.7, 0.85] # 0.75, 85
probs_mate = [0.5]
probs_mutate = [ 0.03, 0.09]

regist = gaMLP_ragnarok(
    mutations = Mutations,
    architecture = architecture,
    pop_sizes = Pop_sizes,
    cross = Cross,
    max_generations= generations,
    tournament_sizes = Tournaments,
    mate_indpb = probs_mate,
    mutate_indpb = probs_mutate,
    classifier = MLPclassifer,
    ineq_measure = Theil,
    ineq_min = 0,
    k_folds=5,
    limit_unchanged = 60,
    X_train = X_train,
    y_train = y_train,
    verbose = True,
)

----------------------------------------
(55, 80, 5, 0.05, 0.7, 0.5, 0.03)
----------------------------------------
---------------
Generation:  1
---------------
Fitest : 0.6986196395750166 Inequality:  0.013301791766174973 Generation without improvement:  0
Stats mean:  0.6088958244741508 std:  0.15357443801524 min: 0.0 max:  0.6986196395750166
0.6986196395750166 0.013301791766174973 0 (0.6088958244741508, 0.15357443801524, 0.0, 0.6986196395750166)
---------------
Generation:  2
---------------
Fitest : 0.6986196395750166 Inequality:  0.01903991898382148 Generation without improvement:  1
Stats mean:  0.5999048105255295 std:  0.17928893966798726 min: 0.0 max:  0.6986196395750166
0.6986196395750166 0.01903991898382148 1 (0.5999048105255295, 0.17928893966798726, 0.0, 0.6986196395750166)
---------------
Generation:  3
---------------
Fitest : 0.7057872794033633 Inequality:  0.0029680748680091405 Generation without improvement:  2
Stats mean:  0.6627997381428697 std:  0.08720168539001931

In [ ]:
parameters_ = select_optimun_parameters(regist=regist,
                           architecture=architecture,
                             k_folds=5,
                             X_train=X_train,
                             y_train=y_train,
                             assessment_metric = recall_score)

In [ ]:
fittest_individual, history, stats = gaMLP_score(
                    classifier = MLPclassifer,
                    architecture = architecture,
                    population_size =parameters_[0],
                    max_generations = parameters_[1],
                    tournament_size =parameters_[2],
                    MUTPB = parameters_[3],
                    CXPB = parameters_[4],
                    mate_indpb = parameters_[5],
                    mutate_indpb = parameters_[6],
                    ineq_measure=Theil,
                    ineq_min  = 0,
                    limit_unchanged = 80,
                    k_folds = 5,
                    X_train = X_train,
                    y_train = y_train,
                    verbose=True)

In [ ]:
preds_GA = np.where(MLP(architecture=architecture, X_train=X_test, y_train=y_test, initial_solution=fittest_individual, preds=True)>0.5,1,0)
GA  = pd.DataFrame(classification_report(y_test, preds_GA, output_dict=True)).iloc[:,0:2]
GA

In [ ]:
from LocalSearch import SA_max
from LocalSearch import search
from sklearn.metrics import f1_score
SAsolution  = SA_max(
    solution = fittest_individual,
    search = search,
    Tf= 0,
    cooling_rate = 75,
    beta = 0.001,
    reduce_temp=0.00001,  #0.0001
    seed_random= 123,
    alpha = 0.1,
    max_iterations = 150, #100
    fitnessFunction = lambda x: MLP(architecture=architecture,
                                       X_train = X_train,
                                       y_train = y_train,
                                       initial_solution=x,
                                       score = f1_score))

In [ ]:
preds_GASA = np.where(MLP(architecture=architecture, X_train=X_test, y_train=y_test,
                          initial_solution=SAsolution['major'][1],
                          preds=True)>0.5,1,0)
GASA  = pd.DataFrame(classification_report(y_test, preds_GASA, output_dict=True)).iloc[:,0:2]


In [ ]:
models_tab = pd.concat([lr_table, SVM_table, MLP_table, GASA], axis=1)
cols_names =  pd.MultiIndex.from_tuples([('Logistic regression','No-churn'),("Logistic regression",'Churn'),
              ("Support vector machine",'No-churn'),('Support vector machine','churn'),
              ('Backpropagation NN', 'No-churn'), ('Backpropagation NN', 'churn'),
              ('Genetic annealing NN', 'No-churn'), ('Genetic annealing NN', 'churn'),])
models_tab.columns  = cols_names
models_tab.to_excel("resultados.xlsx")
models_tab